# Intro

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, pearsonr
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.metrics import roc_auc_score, cohen_kappa_score
import random
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, r2_score, mean_squared_error
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
%matplotlib inline
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

#Dataset imported

In [2]:
df =  pd.read_csv("data/EMLyon_data.csv")

In [3]:
df.columns

Index(['year', 'month', 'week_ref', 'code_market', 'market', 'business_unit',
       'code_resort', 'resort', 'sector_resort', 'client_typo', 'loyal_new',
       'nbr_gm'],
      dtype='object')

###Info & Description

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225424 entries, 0 to 225423
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   year           225424 non-null  int64 
 1   month          225424 non-null  int64 
 2   week_ref       225424 non-null  object
 3   code_market    225424 non-null  int64 
 4   market         225424 non-null  object
 5   business_unit  225424 non-null  object
 6   code_resort    225422 non-null  object
 7   resort         225422 non-null  object
 8   sector_resort  225422 non-null  object
 9   client_typo    225424 non-null  object
 10  loyal_new      225424 non-null  object
 11  nbr_gm         225424 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 20.6+ MB


In [5]:
working_copy = df.copy()

In [6]:
working_copy.head()

,year,month,week_ref,code_market,market,business_unit,code_resort,resort,sector_resort,client_typo,loyal_new,nbr_gm
0,2021,11,2021-11-13,30,SWITZERLAND,EAF,ALBC,ALBION,EUROPE AFRICA SEA & COUNTRYSIDE,Adults,Loyal,11
1,2021,11,2021-11-13,50,GERMANY,EAF,ALBC,ALBION,EUROPE AFRICA SEA & COUNTRYSIDE,Family,Loyal,4
2,2021,11,2021-11-13,20,BELGIUM,EAF,ALBC,ALBION,EUROPE AFRICA SEA & COUNTRYSIDE,Adults,New,6
3,2021,11,2021-11-13,130,SOUTH AFRICA,EAF,ALBC,ALBION,EUROPE AFRICA SEA & COUNTRYSIDE,Family,New,6
4,2021,11,2021-11-13,10,FRANCE,EAF,ALBC,ALBION,EUROPE AFRICA SEA & COUNTRYSIDE,Family,Loyal,15


### Sorted by chronological order

In [7]:
working_copy.sort_values(by=["week_ref"], inplace=True, ascending=True)

In [8]:
working_copy.head()

,year,month,week_ref,code_market,market,business_unit,code_resort,resort,sector_resort,client_typo,loyal_new,nbr_gm
100443,2017,12,2017-12-30,330,BRAZIL,SOUTH AMERICA,LAPC,LAKE PARADISE,SOUTH AMERICA,Adults,Loyal,20
100702,2017,12,2017-12-30,370,CHINA MAINLAND,CH-ML,TOMC,TOMAMU,ASIA MOUNTAIN,Family,New,6
100701,2017,12,2017-12-30,190,AUSTRALIA,ESAP,TOMC,TOMAMU,ASIA MOUNTAIN,Family,New,49
100700,2017,12,2017-12-30,210,SINGAPORE,ESAP,TOMC,TOMAMU,ASIA MOUNTAIN,Family,New,10
100699,2017,12,2017-12-30,190,AUSTRALIA,ESAP,TOMC,TOMAMU,ASIA MOUNTAIN,Adults,New,2


# Data Cleaning
---
1. Clean Duplicates and Missing Values
2. Merge La Rosière
3. Drop Closed Resorts


### 1. Clean Duplicates and Missing Values
---

In [9]:
def duplicate_data(data):

    duplicates_cells = data[data.duplicated].shape

    return f"Number of duplicates: {duplicates_cells[0]}"

### No duplicates

In [10]:
def missing_values(data):

    missing_cells = data.isna().sum().sum()

    return f"Missing values: {missing_cells:.2f}"


In [11]:
duplicate_data(working_copy)

'Number of duplicates: 0'

In [12]:
missing_values(working_copy)

'Missing values: 6.00'

In [13]:
df[df.isna().any(axis=1)]

,year,month,week_ref,code_market,market,business_unit,code_resort,resort,sector_resort,client_typo,loyal_new,nbr_gm
166882,2018,3,2018-03-10,10,FRANCE,EAF,NaN,NaN,NaN,Adults,Loyal,2
213205,2019,1,2019-01-12,80,USA,NORTH AMERICA,NaN,NaN,NaN,Adults,New,4


In [14]:
df.dropna(inplace=True)

### 6 missing values in 2 inputs, that do not seem to be linked. Deleted.

In [15]:
missing_values(df)

'Missing values: 0.00'

### 2. Merge La Rosière
---

### Resort "LA ROSIERE" has 2 "code_resort" denominations, LROV & LROC. LROV minimal, no particular sense. LROV to be converted to LROC

In [16]:
unique_options = working_copy.nunique()
print("Number of unique options in every column:")
print(unique_options)

Number of unique options in every column:
year               7
month             12
week_ref         314
code_market       50
market            50
business_unit      5
code_resort       93
resort            92
sector_resort      9
client_typo        2
loyal_new          2
nbr_gm           885
dtype: int64


In [17]:
working_copy[working_copy['resort']== "LA ROSIERE"].code_resort.value_counts()

LROC    1418
LROV      25
Name: code_resort, dtype: int64

In [18]:
working_copy[working_copy['code_resort']== "LROV"].head()

,year,month,week_ref,code_market,market,business_unit,code_resort,resort,sector_resort,client_typo,loyal_new,nbr_gm
221142,2023,12,2023-12-16,130,SOUTH AFRICA,EAF,LROV,LA ROSIERE,EUROPE MOUNTAIN,Family,New,4
221143,2023,12,2023-12-16,70,UNITED KINGDOM,EAF,LROV,LA ROSIERE,EUROPE MOUNTAIN,Family,New,12
221144,2023,12,2023-12-16,10,FRANCE,EAF,LROV,LA ROSIERE,EUROPE MOUNTAIN,Family,Loyal,4
221145,2023,12,2023-12-16,70,UNITED KINGDOM,EAF,LROV,LA ROSIERE,EUROPE MOUNTAIN,Family,Loyal,9
221146,2023,12,2023-12-16,160,RUSSIA,EAF,LROV,LA ROSIERE,EUROPE MOUNTAIN,Family,New,7


In [19]:
working_copy['code_resort'] = working_copy['code_resort'].replace("LROV", "LROC")

In [20]:
working_copy[working_copy['resort']== "LA ROSIERE"].code_resort.value_counts()

LROC    1443
Name: code_resort, dtype: int64

In [21]:
duplicate_data(working_copy)

'Number of duplicates: 1'

In [22]:
duplicates = working_copy.duplicated(keep=False)

duplicate_rows = working_copy[duplicates]
duplicate_rows.head()

,year,month,week_ref,code_market,market,business_unit,code_resort,resort,sector_resort,client_typo,loyal_new,nbr_gm
2218,2023,12,2023-12-23,80,USA,NORTH AMERICA,LROC,LA ROSIERE,EUROPE MOUNTAIN,Family,New,3
2170,2023,12,2023-12-23,80,USA,NORTH AMERICA,LROC,LA ROSIERE,EUROPE MOUNTAIN,Family,New,3


### An LROC seems to have been recorded as a LROC also. Deleted

In [23]:
group_columns = working_copy.columns.difference(['nbr_gm']).tolist()

working_copy = working_copy.groupby(group_columns, as_index=False)['nbr_gm'].sum()


In [24]:
working_copy.dtypes

business_unit    object
client_typo      object
code_market       int64
code_resort      object
loyal_new        object
market           object
month             int64
resort           object
sector_resort    object
week_ref         object
year              int64
nbr_gm            int64
dtype: object

In [25]:
working_copy[(working_copy['market'] == 'USA') &
             (working_copy['code_resort'] == 'LROC') &
             (working_copy['client_typo'] == 'Family') &
             (working_copy['loyal_new'] == 'New') &
             (working_copy['week_ref']=="2023-12-23")]


,business_unit,client_typo,code_market,code_resort,loyal_new,market,month,resort,sector_resort,week_ref,year,nbr_gm
201772,NORTH AMERICA,Family,80,LROC,New,USA,12,LA ROSIERE,EUROPE MOUNTAIN,2023-12-23,2023,6


In [26]:
duplicate_data(working_copy)

'Number of duplicates: 0'

In [27]:
working_copy.head()

,business_unit,client_typo,code_market,code_resort,loyal_new,market,month,resort,sector_resort,week_ref,year,nbr_gm
0,CH-ML,Adults,370,AEXC,New,CHINA MAINLAND,12,ARCS EXTREME,EUROPE MOUNTAIN,2018-12-15,2018,2
1,CH-ML,Adults,370,AEXC,New,CHINA MAINLAND,12,ARCS EXTREME,EUROPE MOUNTAIN,2019-12-14,2019,1
2,CH-ML,Adults,370,ALBC,Loyal,CHINA MAINLAND,1,ALBION,EUROPE AFRICA SEA & COUNTRYSIDE,2018-01-27,2018,9
3,CH-ML,Adults,370,ALBC,Loyal,CHINA MAINLAND,1,ALBION,EUROPE AFRICA SEA & COUNTRYSIDE,2019-01-26,2019,6
4,CH-ML,Adults,370,ALBC,Loyal,CHINA MAINLAND,1,ALBION,EUROPE AFRICA SEA & COUNTRYSIDE,2020-01-04,2020,2


### 3. Drop Closed Resorts
---

In [28]:
def get_years_for_each_resort(df):
    # Ensure 'year' is an integer
    df['year'] = df['year'].astype(int)

    # Group by resort and get unique years
    grouped = df.groupby('resort')['year'].unique()

    # Convert the Series to a dictionary
    resort_years_dict = grouped.to_dict()

    # Optionally, convert numpy arrays to lists
    resort_years_dict = {resort: years.tolist() for resort, years in resort_years_dict.items()}

    return resort_years_dict

def print_resort_years(resort_years_dict):
    for resort, years in resort_years_dict.items():
        years_str = ", ".join(map(str, years))
        print(f"Resort: {resort} - Years: {years_str}")

print_resort_years(get_years_for_each_resort(working_copy))

Resort: AGADIR - Years: 2019, 2018, 2020, 2017
Resort: AIME LA PLAGNE - Years: 2018, 2019, 2020, 2017
Resort: ALBION - Years: 2018, 2019, 2020, 2023, 2017, 2022, 2021
Resort: ALPE D'HUEZ - Years: 2023, 2022, 2020, 2021, 2019
Resort: ALPE D'HUEZ LA SARENNE - Years: 2018, 2019, 2017
Resort: ANJI - Years: 2018, 2020, 2022, 2023, 2019, 2021
Resort: ARCS EXTREME - Years: 2018, 2019, 2020, 2022, 2023, 2017, 2021
Resort: ARCS PANORAMA - Years: 2019, 2023, 2022, 2021, 2018, 2020
Resort: AVORIAZ - Years: 2018, 2017
Resort: BALI - Years: 2018, 2019, 2020, 2023, 2022, 2017
Resort: BEIDAHU - Years: 2018, 2019, 2020, 2021, 2022, 2023, 2017
Resort: BINTAN ISLAND - Years: 2018, 2019, 2020, 2023, 2022, 2017
Resort: BODRUM PALMIYE - Years: 2023, 2019, 2022, 2018, 2021
Resort: CANCUN - Years: 2018, 2022, 2019, 2020, 2021, 2023, 2017
Resort: CAP SKIRRING - Years: 2018, 2019, 2020, 2022, 2023, 2017, 2021
Resort: CARGESE - Years: 2019, 2018
Resort: CEFALU - Years: 2023, 2022, 2019, 2018, 2021, 2020
Resort:

In [29]:
def drop_closed_resorts(df, recent_years=1):
    # Ensure 'year' is an integer
    df['year'] = df['year'].astype(int)

    # Determine the cutoff year - resorts must have data after this year to be considered open
    cutoff_year = df['year'].max() - recent_years

    # Group by resort and find the max year of data for each resort
    latest_year_per_resort = df.groupby('code_resort')['year'].max()

    # Identify resorts considered open
    open_resorts = latest_year_per_resort[latest_year_per_resort > cutoff_year].index
    closed_resorts = latest_year_per_resort[latest_year_per_resort <=  cutoff_year].index

    # Filter the dataframe to keep only open resorts
    filtered_df = df[df['code_resort'].isin(open_resorts)]

    return filtered_df, open_resorts , closed_resorts, len(open_resorts), len(closed_resorts)


filtered_df, open_resorts ,closed_resorts,nbr_open, nbr_closed= drop_closed_resorts(working_copy, recent_years=1)
print(f"Closed Resorts: {closed_resorts}\n")
print(f"Number of Open Resorts: {nbr_open} \n")
print(f"Number of Closed Resorts: {nbr_closed} \n")

Closed Resorts: Index(['AGAC', 'AHSC', 'APLC', 'AVOC', 'CERC', 'CHAC', 'CRGC', 'DALC', 'ITAC',
       'KAHC', 'KAMC', 'KEMC', 'MANC', 'MCHC', 'NAPC', 'SAMC', 'SANC', 'SAYC',
       'TVCC', 'VISC', 'VLPC', 'VSOC'],
      dtype='object', name='code_resort')

Number of Open Resorts: 70 

Number of Closed Resorts: 22 


### Decision to keep resorts only that contain records for the year 2022. 22 resorts laid off. 70 kept.

In [30]:
print_resort_years(get_years_for_each_resort(filtered_df))

Resort: ALBION - Years: 2018, 2019, 2020, 2023, 2017, 2022, 2021
Resort: ALPE D'HUEZ - Years: 2023, 2022, 2020, 2021, 2019
Resort: ANJI - Years: 2018, 2020, 2022, 2023, 2019, 2021
Resort: ARCS EXTREME - Years: 2018, 2019, 2020, 2022, 2023, 2017, 2021
Resort: ARCS PANORAMA - Years: 2019, 2023, 2022, 2021, 2018, 2020
Resort: BALI - Years: 2018, 2019, 2020, 2023, 2022, 2017
Resort: BEIDAHU - Years: 2018, 2019, 2020, 2021, 2022, 2023, 2017
Resort: BINTAN ISLAND - Years: 2018, 2019, 2020, 2023, 2022, 2017
Resort: BODRUM PALMIYE - Years: 2023, 2019, 2022, 2018, 2021
Resort: CANCUN - Years: 2018, 2022, 2019, 2020, 2021, 2023, 2017
Resort: CAP SKIRRING - Years: 2018, 2019, 2020, 2022, 2023, 2017, 2021
Resort: CEFALU - Years: 2023, 2022, 2019, 2018, 2021, 2020
Resort: CHANGBAISHAN - Years: 2022, 2023
Resort: CHERATING BEACH - Years: 2018, 2019, 2020, 2023, 2017, 2022, 2021
Resort: CLUB MED TWO - Years: 2018, 2019, 2020, 2022, 2023, 2021, 2017
Resort: COLUMBUS ISLE - Years: 2018, 2019, 2020, 202

In [31]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200300 entries, 0 to 225399
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   business_unit  200300 non-null  object
 1   client_typo    200300 non-null  object
 2   code_market    200300 non-null  int64 
 3   code_resort    200300 non-null  object
 4   loyal_new      200300 non-null  object
 5   market         200300 non-null  object
 6   month          200300 non-null  int64 
 7   resort         200300 non-null  object
 8   sector_resort  200300 non-null  object
 9   week_ref       200300 non-null  object
 10  year           200300 non-null  int32 
 11  nbr_gm         200300 non-null  int64 
dtypes: int32(1), int64(3), object(8)
memory usage: 19.1+ MB


In [32]:
filtered_df.to_csv("data/clean_dataset.csv")